In [50]:
import numpy as np
import scipy.stats as st


def skewness(x: np.ndarray):
    x_mu = x - x.mean()
    sigma_hat = np.sqrt( np.mean(x_mu**2) )
    
    return np.mean( x_mu**3 ) / sigma_hat**3

def skewness2(A: np.ndarray) -> np.ndarray:
    c_num = A.shape[1]

    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mu = A - A.mean(axis=1)[:, None]

    # Sigma across rows
    sigma_A = np.sqrt( np.einsum('ij,ij->i', A_mu, A_mu) / c_num )

    return 1/c_num * np.einsum('ij,ij,ij->i', A_mu, A_mu, A_mu) / sigma_A**3

def cis(boot: np.ndarray, theta_hat: float, se_boot: float, alpha: float = 0.05):
    z = st.norm.ppf(1 - alpha/2)

    quantile_low = np.quantile(boot, alpha/2)
    quantile_top = np.quantile(boot, 1 - alpha/2)

    # Normal CI
    normal_ci = (theta_hat - z*se_boot, theta_hat + z*se_boot)

    # Percentile CI
    percentile_ci = (quantile_low, quantile_top)

    # Pivotal CI
    pivotal_ci = (2*theta_hat - quantile_top, 2*theta_hat - quantile_low)

    return normal_ci, percentile_ci, pivotal_ci

n = 50
B = 1000

Y = st.norm.rvs(size=n, random_state=42)
X = np.exp(Y)

theta_hat = st.skew(X) # or skewness(X)
print(theta_hat)

# Bootstrap
rng = np.random.default_rng(42)
xx = rng.choice(X, size=(B,n), replace=True)

skew_boot = skewness2(xx)
se_boot = skew_boot.std()
print(se_boot)

# Confidence intervals:
for ci in cis(skew_boot, theta_hat, se_boot):
    print(ci)

2.226959915173402
0.483657127824893
(1.2790093637705264, 3.1749104665762773)
(1.3297228178532137, 3.2386798688082434)
(1.2152399615385603, 3.12419701249359)


In [51]:
from tqdm.notebook import tqdm

def coverage(band: np.ndarray, true_value: float) -> float:
    return np.mean(np.logical_and(band[0, :] <= true_value, band[1, :] >= true_value))
    
skew_true = (np.exp(1) + 2)*np.sqrt(np.exp(1)-1)

n_exp = 1000

normal_band = np.zeros((2, n_exp))
percentile_band = np.zeros((2, n_exp))
pivot_band = np.zeros((2, n_exp))

for i in tqdm(range(n_exp)):
    Y = st.norm.rvs(size=n, random_state=i)
    X = np.exp(Y)

    theta_hat = st.skew(X) # or skewness(X)
    
    # Bootstrap
    # rng = np.random.default_rng(i)
    # xx = rng.choice(X, size=(B,n), replace=True)
    xx = np.random.choice(X, size=(B,n), replace=True)

    skew_boot = skewness2(xx)
    se_boot = skew_boot.std()

    # Confidence intervals
    all_cis = cis(skew_boot, theta_hat, se_boot)

    normal_band[:, i] = all_cis[0]
    percentile_band[:, i] = all_cis[1]
    pivot_band[:, i] = all_cis[2]

ci_bands = [
    normal_band,
    percentile_band,
    pivot_band
]

for ci_band in ci_bands:
    print(coverage(ci_band, skew_true))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.139
0.027
0.175
